# Spark SQL e Data Frames

#### [Baseado em "Introduction to Spark with Python, by Jose A. Dianes"](https://github.com/jadianes/spark-py-notebooks)

Este notebook apresentará os recursos do Spark para lidar com dados de maneira estruturada. Basicamente, tudo gira em torno do conceito de *Data Frame* e usando *SQL language* para consultá-los. Veremos como a abstração do quadro de dados, muito popular em outros ecossistemas de análise de dados (por exemplo, R e Python/Pandas), é muito poderosa ao executar a análise exploratória de dados. De fato, é muito fácil expressar consultas de dados quando usadas em conjunto com a linguagem SQL. Além disso, o Spark distribui essa estrutura de dados baseada em colunas de forma transparente, a fim de tornar o processo de consulta o mais eficiente possível.

## Fazendo download do Dataset

Neste notebook, usaremos o conjunto de dados reduzido (10 por cento) fornecido para a KDD Cup 1999, contendo quase meio milhão de interações de rede. O arquivo é fornecido como um arquivo *Gzip* que será baixado localmente.

In [1]:
from urllib.request import urlretrieve

f = urlretrieve("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz", "kddcup.data_10_percent.gz")

## Obtendo os dados e criando o RDD

Como fizemos nos cadernos anteriores, usaremos o conjunto de dados reduzido (10%) fornecido para a [KDD Cup 1999](http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html), contendo quase a metade milhões de interações nework. O arquivo é fornecido como um arquivo Gzip que será baixado localmente.

In [2]:
data_file = "./kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file).cache()

## Obtendo um quadro de dados

Um Spark `DataFrame` é uma coleção distribuída de dados organizados em colunas nomeadas. É conceitualmente equivalente a uma tabela em um banco de dados relacional ou a um quadro de dados em R ou Pandas. Eles podem ser construídos a partir de uma ampla variedade de fontes, como um RDD existente no nosso caso.

O ponto de entrada em toda a funcionalidade do SQL no Spark é a classe `SQLContext`. Para criar uma instância básica, tudo o que precisamos é de uma referência ao `SparkContext`. Como estamos executando o Spark no modo shell (usando pySpark), podemos usar o objeto de contexto global `sc` para essa finalidade.

In [3]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

### Inferir o esquema

Com um `SQLContext`, estamos prontos para criar um `DataFrame` a partir do nosso RDD existente. Mas primeiro precisamos informar ao Spark SQL o esquema em nossos dados.

O Spark SQL pode converter um RDD de objetos `Row` em um` DataFrame`. As linhas são construídas passando uma lista de pares chave/valor como *kwargs* para a classe `Row`. As chaves definem os nomes das colunas e os tipos são inferidos observando a primeira linha. Portanto, é importante que não haja dados ausentes na primeira linha do RDD para inferir corretamente o esquema.

No nosso caso, primeiro precisamos dividir os dados separados por vírgula e, em seguida, usar as informações na descrição da tarefa do KDD de 1999 para obter os [column names](http://kdd.ics.uci.edu/databases/kddcup99/kddcup.names).

In [4]:
from pyspark.sql import Row

csv_data = raw_data.map(lambda l: l.split(","))
row_data = csv_data.map(lambda p: Row(
    duration=int(p[0]), 
    protocol_type=p[1],
    service=p[2],
    flag=p[3],
    src_bytes=int(p[4]),
    dst_bytes=int(p[5])
    )
)

Uma vez que tenhamos o nosso RDD de `Row`, podemos inferir e registrar o esquema.

In [5]:
interactions_df = sqlContext.createDataFrame(row_data)
interactions_df.registerTempTable("interactions")

Agora podemos executar consultas SQL sobre nosso quadro de dados que foi registrado como uma tabela.

In [6]:
# Select tcp network interactions with more than 1 second duration and no transfer from destination
tcp_interactions = sqlContext.sql("""
    SELECT duration, dst_bytes FROM interactions WHERE protocol_type = 'tcp' AND duration > 1000 AND dst_bytes = 0
""")
tcp_interactions.show()

+--------+---------+
|duration|dst_bytes|
+--------+---------+
|    5057|        0|
|    5059|        0|
|    5051|        0|
|    5056|        0|
|    5051|        0|
|    5039|        0|
|    5062|        0|
|    5041|        0|
|    5056|        0|
|    5064|        0|
|    5043|        0|
|    5061|        0|
|    5049|        0|
|    5061|        0|
|    5048|        0|
|    5047|        0|
|    5044|        0|
|    5063|        0|
|    5068|        0|
|    5062|        0|
+--------+---------+
only showing top 20 rows



Os resultados das consultas SQL são RDDs e suportam todas as operações RDD normais.

In [7]:
# Output duration together with dst_bytes
tcp_interactions_out = tcp_interactions.rdd.map(lambda p: "Duration: {}, Dest. bytes: {}".format(p.duration, p.dst_bytes))
for ti_out in tcp_interactions_out.collect():
  print(ti_out)

Duration: 5057, Dest. bytes: 0
Duration: 5059, Dest. bytes: 0
Duration: 5051, Dest. bytes: 0
Duration: 5056, Dest. bytes: 0
Duration: 5051, Dest. bytes: 0
Duration: 5039, Dest. bytes: 0
Duration: 5062, Dest. bytes: 0
Duration: 5041, Dest. bytes: 0
Duration: 5056, Dest. bytes: 0
Duration: 5064, Dest. bytes: 0
Duration: 5043, Dest. bytes: 0
Duration: 5061, Dest. bytes: 0
Duration: 5049, Dest. bytes: 0
Duration: 5061, Dest. bytes: 0
Duration: 5048, Dest. bytes: 0
Duration: 5047, Dest. bytes: 0
Duration: 5044, Dest. bytes: 0
Duration: 5063, Dest. bytes: 0
Duration: 5068, Dest. bytes: 0
Duration: 5062, Dest. bytes: 0
Duration: 5046, Dest. bytes: 0
Duration: 5052, Dest. bytes: 0
Duration: 5044, Dest. bytes: 0
Duration: 5054, Dest. bytes: 0
Duration: 5039, Dest. bytes: 0
Duration: 5058, Dest. bytes: 0
Duration: 5051, Dest. bytes: 0
Duration: 5032, Dest. bytes: 0
Duration: 5063, Dest. bytes: 0
Duration: 5040, Dest. bytes: 0
Duration: 5051, Dest. bytes: 0
Duration: 5066, Dest. bytes: 0
Duration

Podemos facilmente dar uma olhada no nosso esquema de quadros de dados usando o `printSchema`.

In [8]:
interactions_df.printSchema()

root
 |-- dst_bytes: long (nullable = true)
 |-- duration: long (nullable = true)
 |-- flag: string (nullable = true)
 |-- protocol_type: string (nullable = true)
 |-- service: string (nullable = true)
 |-- src_bytes: long (nullable = true)



## Consultas como operações `DataFrame`

O Spark `DataFrame` fornece uma linguagem específica de domínio para manipulação de dados estruturados. Essa linguagem inclui métodos que podemos concatenar para fazer seleção, filtragem, agrupamento, etc. Por exemplo, digamos que queremos contar quantas interações existem para cada tipo de protocolo. Nós podemos proceder da seguinte forma.

In [9]:
from time import time

t0 = time()
interactions_df.select("protocol_type", "duration", "dst_bytes").groupBy("protocol_type").count().show()
tt = time() - t0

print("Query performed in {} seconds".format(round(tt,3)))

+-------------+------+
|protocol_type| count|
+-------------+------+
|          tcp|190065|
|          udp| 20354|
|         icmp|283602|
+-------------+------+

Query performed in 11.984 seconds


Agora imagine que queremos contar quantas interações duram mais de 1 segundo, sem transferência de dados do destino, agrupadas por tipo de protocolo. Podemos apenas adicionar para filtrar chamadas para o anterior.

In [10]:
t0 = time()
interactions_df.select("protocol_type", "duration", "dst_bytes").filter(interactions_df.duration>1000).filter(interactions_df.dst_bytes==0).groupBy("protocol_type").count().show()
tt = time() - t0

print("Query performed in {} seconds".format(round(tt,3)))

+-------------+-----+
|protocol_type|count|
+-------------+-----+
|          tcp|  139|
+-------------+-----+

Query performed in 14.18 seconds


Podemos usar isso para realizar algumas [exploratory data analysis](http://en.wikipedia.org/wiki/Exploratory_data_analysis). Vamos contar quantas interações normais e de ataque nós temos. Primeiro, precisamos adicionar a coluna de rótulo aos nossos dados.    

In [11]:
def get_label_type(label):
    if label!="normal.":
        return "attack"
    else:
        return "normal"
    
row_labeled_data = csv_data.map(lambda p: Row(
    duration=int(p[0]), 
    protocol_type=p[1],
    service=p[2],
    flag=p[3],
    src_bytes=int(p[4]),
    dst_bytes=int(p[5]),
    label=get_label_type(p[41])
    )
)
interactions_labeled_df = sqlContext.createDataFrame(row_labeled_data)

Desta vez, não precisamos registrar o esquema, pois vamos usar a interface de consulta OO.

Vamos verificar o anterior realmente funciona contando dados de ataque e normal em nosso quadro de dados.

In [12]:
t0 = time()
interactions_labeled_df.select("label").groupBy("label").count().show()
tt = time() - t0

print("Query performed in {} seconds".format(round(tt,3)))

+------+------+
| label| count|
+------+------+
|normal| 97278|
|attack|396743|
+------+------+

Query performed in 9.763 seconds


Agora queremos contá-las por rótulo e tipo de protocolo, para ver o quão importante é o tipo de protocolo detectar quando uma interação é ou não um ataque.

In [13]:
t0 = time()
interactions_labeled_df.select("label", "protocol_type").groupBy("label", "protocol_type").count().show()
tt = time() - t0

print("Query performed in {} seconds".format(round(tt,3)))

+------+-------------+------+
| label|protocol_type| count|
+------+-------------+------+
|normal|          udp| 19177|
|normal|         icmp|  1288|
|normal|          tcp| 76813|
|attack|         icmp|282314|
|attack|          tcp|113252|
|attack|          udp|  1177|
+------+-------------+------+

Query performed in 8.901 seconds


À primeira vista, parece que as interações *udp* estão em menor proporção entre ataques de rede e outros tipos de protocolos.

E podemos fazer agrupamentos muito mais sofisticados. Por exemplo, adicione ao anterior um "split" baseado na transferência de dados do destino.

In [14]:
t0 = time()
interactions_labeled_df.select("label", "protocol_type", "dst_bytes").groupBy("label", "protocol_type", interactions_labeled_df.dst_bytes==0).count().show()
tt = time() - t0

print("Query performed in {} seconds".format(round(tt,3)))

+------+-------------+---------------+------+
| label|protocol_type|(dst_bytes = 0)| count|
+------+-------------+---------------+------+
|normal|          udp|          false| 15583|
|attack|          udp|          false|    11|
|attack|          tcp|           true|110583|
|normal|          tcp|          false| 67500|
|attack|         icmp|           true|282314|
|attack|          tcp|          false|  2669|
|normal|          tcp|           true|  9313|
|normal|          udp|           true|  3594|
|normal|         icmp|           true|  1288|
|attack|          udp|           true|  1166|
+------+-------------+---------------+------+

Query performed in 10.764 seconds


Vemos quão relevante é essa nova divisão para determinar se uma interação de rede é um ataque.

Vamos parar por aqui, mas podemos ver como esse tipo de consulta é poderosa para explorar nossos dados. Na verdade, podemos replicar todas as divisões que vimos nos cadernos anteriores, ao introduzir árvores de classificação, apenas selecionando, tateando e filtrando nosso dataframe. Para uma lista mais detalhada (mas menos real) das operações e fontes de dados do DataFrame do Spark, dê uma olhada na documentação oficial [aqui](https://spark.apache.org/docs/latest/sql-programming-guide.html#dataframe-operations). 